# SI649-20w Lab 4 -> Altair III
School of Information, University of Michigan


## Assignment Overview
1. Interaction 
2. Review transform 

We will replicate 3 visualizations (and a bonus visualization) created by the article posted on [Five Thirty Eight](https://fivethirtyeight.com) available  [online](https://fivethirtyeight.com/features/the-mayweather-mcgregor-fight-as-told-through-emojis/) (Hickey, Koeze, Dottle, Wezerek 2017). 

**For this lab, please write Altair code to answer the questions. It's fine if your visualization looks slightly different from the example (e.g., getting 1.1 instead of 1.0, use orange instead of red, have different titles, chart width/height,and mark size/opacity)**

**Also, it's possible that your chart does not show the desired interactivity because of altair/colab issues. Check the potential bug section for more detail.**

### Resources:
- Article by [Five Thirty Eight](https://fivethirtyeight.com) available  [online] (https://fivethirtyeight.com/features/the-mayweather-mcgregor-fight-as-told-through-emojis/) (Hickey, Koeze, Dottle, Wezerek 2017)  
- the original can be found on [Five Thirty Eight Mayweather vs McGregor] (https://github.com/fivethirtyeight/data/tree/master/mayweather-mcgregor)

### General Hints: 
* Yes, you can render emojis in colab and in your chart. You can consider them as text. 
* We recommend that you finish all the static charts before adding interactions. 
* If you see duplicated axes, use `axis=None` to get rid of unnecessary axes.  
* Don't forget to set `empty="none"`. The default behavior is that when nothing is selected, *everything* is selected. When set to none, empty selections contain no data values.  
* `resolve_scale` ensures charts share axes and scales. 


In [0]:
# start with the setup
import pandas as pd
import altair as alt
import numpy as np


In [0]:
#load data 
df1=pd.read_csv("https://raw.githubusercontent.com/LiciaHe/SI649/master/week4/data/df1.csv")
df2=pd.read_csv("https://raw.githubusercontent.com/LiciaHe/SI649/master/week4/data/df2_count.csv")
df3=pd.read_csv("https://raw.githubusercontent.com/LiciaHe/SI649/master/week4/data/df3.csv")
df4=pd.read_csv("https://raw.githubusercontent.com/LiciaHe/SI649/master/week4/data/df4.csv")

## Visualization 1: Emoji and percentage of usage 

We will replicate the following visualization  
![vis1_static](https://github.com/LiciaHe/SI649/blob/master/week4/images/vis1/static.png?raw=true)

**Description of the visualization (static):**
*   Use *df1* for this exercise
*   This visualization has 3 components: **emoji**, **percentage text**, and **bars** 
*   All 3 components share the same y axis, which display the *rank* of percentage from highest to lowest. 
*   The width of the bar(along x axis) encodes *PERCENT*
*   All 3 components have a low opacity because we want to add interactions (see the next cell). 


**Description of the visualization (interactivity):**
1. When hovering over bars, the associated emoji will show up as tooltip 
![alt text](https://github.com/LiciaHe/SI649/blob/master/week4/images/vis1/interactive_1.gif?raw=true)
2. When hovering over emojis, percentage texts, or bars, the opacity of the selected row will change to 1. 
![alt text](https://github.com/LiciaHe/SI649/blob/master/week4/images/vis1/interactive_2.gif?raw=true)
3. Brushing over the emojis will filter bars.
![alt text](https://github.com/LiciaHe/SI649/blob/master/week4/images/vis1/interactive_3.gif?raw=true)
4. Brushing over percentage text will filter bars.
![alt text](https://github.com/LiciaHe/SI649/blob/master/week4/images/vis1/interactive_4.gif?raw=true)



**Sample style settings (optional):**
Here's a list of default style settings we used to generate the graph.
* Original opacity for all 3 components: 0.6. 
* Hovered opacity: 1
* bar height = 15, color = orange
* text chart and emoji width are both 20
* after building the compound chart, use the following line to disable border : `.configure_view(strokeWidth=0)`



Hint:
* We recommend getting all static components working before writing any interactivity. 
* Add one interaction at a time and test whether or not it works. 
* To add an interaction that's not tooltip and zooming, you need four steps (review in-class demo). 
* Selection is used in two scenarios: 1) to add to a *condition*, which is used in `encode`. 2) to add in `transform_filter`. In this visualization, you will implement both. Think through which you will use where before trying to build this.

In [43]:
selection=alt.selection_single(on='mouseover',empty='none',)

opacityCondition=alt.condition(selection,alt.value(1),alt.value(0.6))

selection1 = alt.selection_interval()
opacityCondition2=alt.condition(selection1,alt.value(1),alt.value(0))



emoji = alt.Chart(df1).mark_text().encode(
    alt.Y('rank:N',axis=None,
          sort='ascending'),  
          alt.Text('EMOJI'),
          opacity=opacityCondition
          ).add_selection(selection,selection1
                          ).properties(width=20)
    
percent=alt.Chart(df1).mark_text().encode(
    alt.Y('rank:N',axis=None,
          sort='ascending'),
          alt.Text('PERCENT_TEXT'),
          opacity=opacityCondition
          ).add_selection(selection,selection1).properties(width=20)

base=alt.Chart(df1).mark_bar(
    size=15,
    color='orange',
    height=150
    
).encode(
        alt.X('PERCENT',axis=None),
        alt.Y('rank:N',axis=None,
        sort='ascending'),
        opacity=opacityCondition,
        tooltip=alt.Tooltip('EMOJI')
).add_selection(selection1).transform_filter(selection1)


alt.hconcat(emoji,percent,base).resolve_scale(y="shared").configure_view(strokeWidth=0)

alt.HConcatChart(...)

## Visualization 2: Irish Pride vs. Money Team 

We will replicate the following visualization  
![vis2_static](https://github.com/LiciaHe/SI649/blob/master/week4/images/vis2/static.png?raw=true)

**Description of the visualization (static):**
*   Use *df2* 
*   The visualization has 1 "static" component: **line chart**. It displays the relationship between *datetime* and *count* 


**Description of the visualization (interactivity):**
1. Enable zooming and panning along the x-axis. (The gif below only displays the line chart.)
![alt text](https://github.com/LiciaHe/SI649/blob/master/week4/images/vis2/interactive_1.gif?raw=true)
2. Display a vertical line that moves with the mouse. This will require you to add additional chart component(let's call it **vLine**).
![alt text](https://github.com/LiciaHe/SI649/blob/master/week4/images/vis2/interactive_2.gif?raw=true)
3. Display the intersection of the **vLine** with the **line chart** as 2 circles (let's call these two circles **intersection dots**). 
![alt text](https://github.com/LiciaHe/SI649/blob/master/week4/images/vis2/interactive_3.gif?raw=true)
4. When hovering over these **intersection dots**, display the *tweet_count*, *datetime*, and *team* in tooltip.  
![alt text](https://github.com/LiciaHe/SI649/blob/master/week4/images/vis2/interactive_4.gif?raw=true)



**Sample style settings (optional):**
Here's a list of default style settings we used to generate the graph.

* line chart size = 2.5, 
* vLine: size=4, color="lightgray", initial opacity = 0 
* indicator dot:  color="black" size=70

**Bugs**

If your interaction look similar to this, don't worry about it. It's likely a altair/colab issue. 

1. Blinking lines: You can fix by making your **vLine** thicker, or view it in vegalite editor. 

![alt text](https://github.com/LiciaHe/SI649/blob/master/week4/images/vis2/interactive_2_bug1.gif?raw=true)


* If your tooltip does not appear, make sure you have a selection bond to the chart that has the tooltip encoding. 

* If your are getting an error: "avascript Error: Duplicate signal name", it means you added your selections more than once. Remember, when layering charts together, they will share their selections. Therefore, don't add the same selection to both charts. 


**Hint**


* We only want to enable zooming and panning along the x-axis.
*  There are multiple ways of implementing the **vLine**. Here is one of them: 
> 1) use mark_rule to generate a line for every single data point and set these line's opacity to be 0.

 > 2)when mouse hovering over a line, display it by changing its opacity. 

*  The implementation of the **intersection dots** is similar to that of the **vLine**. Do you need a new selection/condition for the **intersection dots**?




In [59]:
selection=alt.selection_interval(bind='scales',encodings=['x'])
selection1=alt.selection_single(encodings=['x'],on='mouseover',empty='none',nearest=True)
selection2=alt.selection_single(encodings=['x'],on='mouseover',empty='none')


c1 = alt.Chart(df2).mark_line().add_selection(selection).encode(
    alt.X('datetime:T',title=''),
    alt.Y('tweet_count:Q',title='Four-minute rolling average'),
    alt.Color('team:N'),
    size=alt.value(2.5),
    #tooltip=['tweet_count','datetime','team']
)

opacityCondition=alt.condition(selection1,alt.value(1),alt.value(0))
vLine=alt.Chart(df2).mark_rule().add_selection(selection1).encode(
    alt.X('datetime:T'),
    size=alt.value(4),
    color=alt.value('lightgray'),
    opacity = opacityCondition,
    tooltip=['tweet_count','datetime:T','team']
)

dots=alt.Chart(df2).mark_circle().add_selection(selection2).encode(
    alt.X('datetime:T'),
    alt.Y('tweet_count:Q',title='Four-minute rolling average'),
    color=alt.condition(selection1,alt.value('black'),alt.value('gray')),
    opacity=alt.condition(selection1,alt.value(1),alt.value(0)),
    size=alt.value(70),
    #tooltip=['tweet_count','datetime','team']
)

c1+vLine+dots

alt.LayerChart(...)

## Visualization 3: Much hype, more boredom

We will replicate the following visualization  
![vis3_static](https://github.com/LiciaHe/SI649/blob/master/week4/images/vis3/static.png?raw=true)

**Description of the visualization (static):**
*   Use *df3*
*   The visualization has 1 component: **line chart**. It displays the relationship between *datetime* and *tweet_count*. Each line represents one emoji. 

**Description of the visualization (interactivity):**
1. Build radio selections for emojis. Theoretically, only one line will be shown at any given time. See the "bug" section for more detail.
![alt text](https://github.com/LiciaHe/SI649/blob/master/week4/images/vis3/interactive_1.gif?raw=true)
2. Brushing over line chart will display individual data points as circles. This will require another chart component, let's call this component **circles**.  
![alt text](https://github.com/LiciaHe/SI649/blob/master/week4/images/vis3/interactive_2.gif?raw=true)



**Sample style settings (optional):**
Here's a list of default style settings we used to generate the graph.

* **circles**: color="black",opacity=0.7 
* For both x and y axis, we have the following tickCount specified
> axis=alt.Axis(tickCount=5,...) 



**Bugs**

If your interaction look similar to this, don't worry about it. It's likely a altair/colab issue. 

1. single click reset selection even when *clear=False*. This is likely a colab issue, because the chart will behave correctly in vega-editor. You can ignore this bug or view it in vegalite online editor. 

![alt text](https://github.com/LiciaHe/SI649/blob/master/week4/images/vis3/interactive_2_bug.gif?raw=true)

**Hint**

* You will have 2 selections. One for the emoji selection and one for the time selection. Ensure that these two interactions work independently before merging them together. 

* You can use multiple transform_filter, or use logical operand to chain multiple selections. 
* There are only 2 emojis in this chart, you can set a variable and refer to it in your radio selection. 
> domain = [ '🔥',  '😴']



In [45]:
df3['id'] =[{'🔥': 1, '😴': 2}[emoji] 
for emoji in df3['emoji'] ]
df3

,Unnamed: 0,datetime,tweet_count,emoji,id
0,0,2017-08-27 00:05:30,0.251824,🔥,1
1,1,2017-08-27 00:05:45,0.520945,🔥,1
2,2,2017-08-27 00:06:00,0.745094,🔥,1
3,3,2017-08-27 00:06:15,0.899476,🔥,1
4,4,2017-08-27 00:06:30,1.020321,🔥,1
...,...,...,...,...,...
551,273,2017-08-27 01:13:45,0.108172,😴,2
552,274,2017-08-27 01:14:00,0.107382,😴,2
553,275,2017-08-27 01:14:15,0.109748,😴,2
554,276,2017-08-27 01:14:30,0.097836,😴,2


In [48]:
genres = df3['emoji'].unique() # get unique field values
genres = list(filter(lambda d: d is not None, genres)) # filter out None values
genres.sort() # sort alphabetically
emo = [genres[0],genres[1]]
em_options=list(df3["id"].unique())
em_options=list(map(lambda x:float(x),em_options))
em_options.sort()

em_radio=alt.binding_radio(
    options=em_options
    )
radio_selector = alt.selection_single(
    name="Select",
    fields=["emoji"],
    clear="click",
    bind={'emoji': alt.binding_radio(options=emo)},
    init={'emoji':genres[0]}
    )

selection=alt.selection_interval(empty="none")
#selection1=alt.selection_single(on='mouseover', nearest=True, clear="click")

colorCondition2=alt.condition(selection,alt.value("black"),alt.value("white"))
opacityCondition = alt.condition(radio_selector, alt.value(0.75), alt.value(0.005))
opacityCondition2 = alt.condition(selection, alt.value(0.75), alt.value(0.005))

c1=alt.Chart(df3, height=400, width=500).mark_line().add_selection(
        radio_selector,
        #selection1
).encode(
    alt.Y("tweet_count:Q",title= "Four-minute rolling avarage",axis=alt.Axis(tickCount=5)),
    alt.X("datetime:T",axis=alt.Axis(tickCount=5)),
    color="emoji",
    opacity = opacityCondition
)

c2 = alt.Chart(df3, height=400, width=500).mark_point().add_selection(
    selection
).encode(
    alt.Y("tweet_count:Q",title= "Four-minute rolling avarage",axis=alt.Axis(tickCount=5)),
    alt.X("datetime:T",axis=alt.Axis(tickCount=5)),
    color=colorCondition2,
    opacity = opacityCondition2
)
#c1+c2
c3= (c1 + c2).add_selection(radio_selector).transform_filter(radio_selector)
c3

alt.LayerChart(...)

## BONUS: Visualization 4: Tears were shed-of joy and sorrow

OPTIONAL:
We will replicate the following visualization  
![vis4_static](https://github.com/LiciaHe/SI649/blob/master/week4/images/vis4/static.png?raw=true)

**Description of the visualization (static):**
*   Use *df4*
*   The visualization has 2 components: **line chart** that displays relationship between *datetime* and *tweet_count*, and **legend** that displays the two emojis in the line chart.  
*   **legend** is a chart. It is not the automatically generated legend. (i.e., In the line chart's color encoding, set *legend=None*)
*   Two components are displayed side by side 

**Description of the visualization (interactivity):**
1. In the legend component, if one of the emoji is clicked, the selected emoji will have full opacity while the other emoji becomes transparent. 
![alt text](https://github.com/LiciaHe/SI649/blob/master/week4/images/vis4/interactive_1.gif?raw=true)
2.  Clicking emojis in the **legend** component will display the corresponding line and hide the other line . 
![alt text](https://github.com/LiciaHe/SI649/blob/master/week4/images/vis4/interactive_2.gif?raw=true)

**Sample style settings (optional):**
Here's a list of default style settings we used to generate the graph.
* For the line chart, we used tickCount=5 in x and y axis.  
* For the legend, we used size=25,strokeWidth=0

**Hint**

* You can have one selection and multiple conditions that use the same selection. 
* To hide a line, you have 2 options: changing opacity and adding transform_filter. The sample answer is generated with the first option. If you use the second option, you will get a slightly different view (because axes will be adjusted) and it's totally ok. 

In [47]:
selection=alt.selection_multi(fields=['emoji'])
opacityCondition1 = alt.condition(selection, alt.value(0.75), alt.value(0.005), legend=None)

c1=alt.Chart(df4,title="Tears were shed-of joy and sorrow", height=400, width=500).mark_line().encode(
    alt.Y("tweet_count:Q",title= "Four-minute rolling avarage",axis=alt.Axis(tickCount=5)),
    alt.X("datetime:T",axis=alt.Axis(tickCount=5),title=""),
    alt.Color('emoji',legend=None),
    opacity = opacityCondition1
)

legend = alt.Chart(df4,title="legend", height=100, width=150
).mark_text(filled=True, size=25,strokeWidth=0
).encode(
    alt.Y('emoji:O', axis=alt.Axis(orient='right',labels=False), title=""),
    alt.SizeValue(37),
    text='emoji',
    opacity = opacityCondition1
).add_selection(
selection
)

c1 | legend

alt.HConcatChart(...)

*This is the end of lab 4*. 


Please run all cells (Runtime->Restart and run all), and 
1.  save to PDF (File->Print->Save PDF -> landscape, shrink to 80%)
2.  save to ipynb (File -> Download .ipynb)

Rename both files with your uniqname: e.g. uniqname.pdf/ uniqname.ipynb

Upload both files to canvas. 


